In [113]:
import schedule
import time
import requests
import pandas as pd
from datetime import datetime, timedelta, date
import pyodbc
from sqlalchemy import create_engine
import mysql.connector

In [114]:
db_config = {
    'host': 'Localhost',
    'user': 'root',
    'password': '8november2002',
    'database': 'bhavDB',
}
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()
# engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")

In [115]:
data=[]
date="02012024"
current_date=datetime.strptime(date,"%d%m%Y") #25-01-2024
counter=0
while(True):
    #Check response and append data
    query=f"SELECT * FROM StockData WHERE DATE1 = '{current_date}'"
    cursor.execute(query)
    result= cursor.fetchall()
    if(len(result)!=0):
        data.append(result)
        counter+=1
        print(f"Open {current_date}")
    else:
        print(f"Close {current_date}")
    current_date-=timedelta(days=1)
    #Break Loop
    if(counter==3):
        break
print(data)

Open 2024-01-02 00:00:00
Open 2024-01-01 00:00:00
Close 2023-12-31 00:00:00
Close 2023-12-30 00:00:00
Open 2023-12-29 00:00:00
[[('20MICRONS', 'EQ', datetime.date(2024, 1, 2), 176.5, 176.5, 177.7, 168.5, 171.5, 171.0, 171.5, 321759, 551.8, 12071, '153090', '47.58'), ('21STCENMGM', 'BE', datetime.date(2024, 1, 2), 32.3, 31.65, 32.55, 31.65, 31.65, 31.65, 31.71, 2296, 0.73, 28, '-', '-'), ('360ONE', 'EQ', datetime.date(2024, 1, 2), 702.85, 695.25, 699.95, 675.0, 676.15, 676.9, 681.25, 348865, 2376.63, 15392, '214297', '61.43'), ('3IINFOLTD', 'EQ', datetime.date(2024, 1, 2), 52.5, 52.85, 60.1, 52.2, 56.9, 56.65, 56.55, 18216265, 10301.8, 56359, '4824691', '26.49'), ('3MINDIA', 'EQ', datetime.date(2024, 1, 2), 36659.9, 36868.9, 36934.2, 35150.1, 35502.0, 35633.9, 35903.1, 9804, 3519.94, 5647, '2010', '20.50'), ('3PLAND', 'EQ', datetime.date(2024, 1, 2), 27.15, 27.9, 27.9, 26.75, 27.2, 27.05, 27.11, 10722, 2.91, 312, '7911', '73.78'), ('5PAISA', 'EQ', datetime.date(2024, 1, 2), 600.8, 604.6

In [116]:
column_names = [desc[0] for desc in cursor.description]
print(column_names)

['SYMBOL', 'SERIES', 'DATE1', 'PREV_CLOSE', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'LAST_PRICE', 'CLOSE_PRICE', 'AVG_PRICE', 'TTL_TRD_QNTY', 'TURNOVER_LACS', 'NO_OF_TRADES', 'DELIV_QTY', 'DELIV_PER']


In [117]:
connection.commit()
cursor.close()
connection.close()

In [118]:
df_today = pd.DataFrame(data[0], columns=column_names)
df_prev = pd.DataFrame(data[1], columns=column_names)
df_prev2 = pd.DataFrame(data[2], columns=column_names)

In [119]:
import pandas as pd

def redSPY(df_today, df_prev, df_prev2):

    symbols = df_today['SYMBOL'].unique().tolist()

    for symbol in symbols:
        today = df_today[df_today['SYMBOL'] == symbol]
        #print(today)
        prev_day = df_prev[df_prev['SYMBOL'] == symbol]
        #print(prev_day)
        prev_day2 = df_prev2[df_prev2['SYMBOL'] == symbol]
        #print(prev_day2)

        # Check if any of the filtered DataFrames is empty
        if today.empty or prev_day.empty or prev_day2.empty:
            continue

        today = today.iloc[0]
        second_day = prev_day.iloc[0]
        first_day = prev_day2.iloc[0]

        # Apply condition to find colors of all three days candles
        second_day_color = 'red' if second_day['OPEN_PRICE'] > second_day['CLOSE_PRICE'] else 'green'
        first_day_color = 'green' if first_day['OPEN_PRICE'] < first_day['CLOSE_PRICE'] else 'red'
        today_color = 'green' if today['OPEN_PRICE'] < today['CLOSE_PRICE'] else 'red'

        # Condition check
        condition_met = (
            first_day_color == 'green' and
            second_day_color == 'red' and
            today_color == 'red' and
            second_day['LOW_PRICE'] > first_day['OPEN_PRICE'] and
            second_day['HIGH_PRICE'] < first_day['CLOSE_PRICE'] and
            today['CLOSE_PRICE'] < first_day['LOW_PRICE']
        )
        # print(f"For Symbol: {symbol, condition_met}")
        # Check if the condition is met and proceed
        if condition_met:
            # entry_price = today['OPEN_PRICE']
            stop_loss = first_day['HIGH_PRICE']
            # target_price = entry_price * 1.1

            print(f"For Symbol: {symbol}")
            # print(f"Sell Signal - Entry Price: Tomorrow Open Price")
            print(f"Stop Loss: {stop_loss}")
            # print(f"Target Price: entry_price * 1.1")
            print()

In [120]:
redSPY(df_today, df_prev, df_prev2)

For Symbol: TIMKEN
Stop Loss: 3264.0

